In [1]:
# Get model coefficients (impact of each channel)
channel_impact = pd.DataFrame({'Channel': X.columns, 'Impact': model.coef_})
channel_impact = channel_impact.sort_values(by='Impact', ascending=False)

print("Estimated Impact of Each Marketing Channel:")
print(channel_impact)


Estimated Impact of Each Marketing Channel:
             Channel    Impact
0                 TV  0.557877
6                SEM  0.315006
1            Digital  0.174909
5         Affiliates  0.137970
7              Radio  0.033879
4   Online marketing -0.066412
2        Sponsorship -0.238816
3  Content Marketing -0.270065
8              Other -0.287042


In [2]:
# ROI = Impact / Spend
roi = channel_impact.copy()
roi['ROI'] = roi['Impact'] / df[X.columns].mean().values  # Divide impact by avg spend per channel
roi = roi.sort_values(by='ROI', ascending=False)

print("Estimated ROI for Each Channel:")
print(roi)



Estimated ROI for Each Channel:
             Channel    Impact       ROI
5         Affiliates  0.137970  0.206233
0                 TV  0.557877  0.150886
6                SEM  0.315006  0.127446
1            Digital  0.174909  0.005745
7              Radio  0.033879  0.002100
4   Online marketing -0.066412 -0.012986
2        Sponsorship -0.238816 -0.031422
8              Other -0.287042 -0.071727
3  Content Marketing -0.270065 -0.693958


In [4]:
# Convert data to time-series format
df.set_index('yearmonth', inplace=True)

# Fit SARIMA Model
sarima_model = sm.tsa.statespace.SARIMAX(df['Total Investment'], order=(1,1,1), seasonal_order=(1,1,1,12))
sarima_result = sarima_model.fit()

# Predict next 12 months
future_dates = pd.period_range(df.index[-1] + 1, periods=12, freq='M')
forecast = sarima_result.forecast(steps=12)

# Store Forecast in DataFrame
forecast_df = pd.DataFrame({'yearmonth': future_dates, 'Forecasted GMV': forecast})
print("\n✅ Forecasted GMV for Next Year:")
print(forecast_df)

Forecasted GMV for Next Year:
        yearmonth  Forecasted GMV
2024-07   2024-07    5.136511e+08
2024-08   2024-08    3.936764e+08
2024-09   2024-09    1.305577e+09
2024-10   2024-10    2.044596e+09
2024-11   2024-11    8.551955e+08
2024-12   2024-12    1.410486e+09
2025-01   2025-01    1.084993e+09
2025-02   2025-02    8.235533e+08
2025-03   2025-03    1.343280e+09
2025-04   2025-04    9.115157e+08
2025-05   2025-05    1.123603e+09
2025-06   2025-06    7.713755e+08


In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from scipy.optimize import minimize
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# Load data
file_path = "/kaggle/input/monthlyyyyy/Monthdata (1).csv"
df = pd.read_csv(file_path)

# Preprocessing
drop_cols = ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Year-Month', 'yearmonth']
df = df.drop(columns=drop_cols)

# Define features and target
X = df.drop(columns=['gmv', 'Total Investment'])
y = df['gmv']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Ridge and Lasso Regression
ridge = Ridge(alpha=1.0).fit(X_train, y_train)
lasso = Lasso(alpha=0.1).fit(X_train, y_train)

ridge_pred = ridge.predict(X_test)
lasso_pred = lasso.predict(X_test)

ridge_score = ridge.score(X_test, y_test)
lasso_score = lasso.score(X_test, y_test)

# 2. Generalized Linear Model (GLM)
glm_model = sm.GLM(y_train, sm.add_constant(X_train), family=sm.families.Gaussian()).fit()
glm_pred = glm_model.predict(sm.add_constant(X_test))
glm_score = -glm_model.aic  # Using AIC for model evaluation

# 3. Bayesian Optimization for Budget Allocation
def objective(params):
    budget_allocation = np.array(params)
    predicted_gmv = ridge.predict([budget_allocation])[0]
    return -predicted_gmv  # Maximizing GMV

space = [Real(0, 1, name=col) for col in X.columns]

@use_named_args(space)
def bayesian_objective(**params):
    return objective(list(params.values()))

res = gp_minimize(bayesian_objective, space, n_calls=20, random_state=42)
best_budget_allocation = res.x

# 4. Quadratic Programming for Constrained Budget Optimization
def quadratic_objective(x):
    return -ridge.predict([x])[0]  # Minimize negative GMV

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Budget constraint
bounds = [(0, 1) for _ in range(len(X.columns))]
res_qp = minimize(quadratic_objective, np.ones(len(X.columns)) / len(X.columns), bounds=bounds, constraints=constraints)
best_qp_allocation = res_qp.x

# Compare results
models_scores = {
    "Ridge": ridge_score,
    "Lasso": lasso_score,
    "GLM (AIC)": glm_score,
    "Bayesian Optimization": -res.fun,
    "Quadratic Programming": -res_qp.fun
}
print("Model Scores:")
for model, score in models_scores.items():
    print(f"{model}: {score}")
best_model = max(models_scores, key=models_scores.get)

print("Best Model:", best_model)
print("Best Bayesian Allocation:", best_budget_allocation)
print("Best Quadratic Allocation:", best_qp_allocation)


Model Scores:
Ridge: -1.2978288343158941
Lasso: -0.5988711140237781
GLM (AIC): 357.2917532945678
Bayesian Optimization: 35690.78148465594
Quadratic Programming: 28390.71039981478
Best Model: Bayesian Optimization
Best Bayesian Allocation:
[1.0, 0.8127469230648015, 1.0, 1.0, 1.0, 0.8977860909342386, 1.0, 1.0, 1.0, 0.0, 1.0, 0.04731259471525188, 1.0, 1.0, 1.0, 0.9691418471100662, 0.27839770402408215, 1.0, 1.0, 1.0, 0.0]
Best Quadratic Allocation:
[0.00000000e+00 5.06889820e-08 1.24680957e-10 1.29802426e-10
 1.05801572e-07 0.00000000e+00 1.55561974e-06 4.18295855e-11
 1.00000000e+00 0.00000000e+00 1.28676738e-06 0.00000000e+00
 0.00000000e+00 1.81077261e-06 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.15934211e-12 0.00000000e+00 0.00000000e+00
 0.00000000e+00]


In [6]:
import pandas as pd
from scipy.stats import ttest_ind

# Assuming your DataFrame is named 'df' and contains 'gmv' and marketing channels
channels = ['TV', 'TV_Ad_Stock', 'Digital', 'Digital_Ad_Stock', 'Sponsorship', 'Sponsorship_Ad_Stock', 'Content Marketing',
     'Content Marketing_Ad_Stock', 'Online marketing', 'Online marketing_Ad_Stock', 'Affiliates',
     'Affiliates_Ad_Stock', 'SEM', 'SEM_Ad_Stock', 'Radio', 'Radio_Ad_Stock', 'Other',
       'Other_Ad_Stock', 'NPS', 'Stock Index']

ttest_results = []

for channel in channels:
    # Create high spend and low spend groups based on median split
    median_spend = df[channel].median()
    high_spend = df[df[channel] > median_spend]['gmv']
    low_spend = df[df[channel] <= median_spend]['gmv']

    # Perform independent T-test
    t_stat, p_val = ttest_ind(high_spend, low_spend, equal_var=False)

    ttest_results.append({
        'Channel': channel,
        'T-Statistic': round(t_stat, 4),
        'P-Value': round(p_val, 4),
        'Significant': 'Yes' if p_val < 0.05 else 'No'
    })

# Convert to DataFrame for better visualization
ttest_df = pd.DataFrame(ttest_results)
print(ttest_df)

T-Statistics and Significance of Channels:
                      Channel  T-Statistic  P-Value Significant
0                           TV       2.4736   0.0184         Yes
1                  TV_Ad_Stock       3.3233   0.0018         Yes
2                      Digital       1.3100   0.1993          No
3             Digital_Ad_Stock       1.0056   0.3210          No
4                  Sponsorship       1.9922   0.0546          No
5         Sponsorship_Ad_Stock       2.2803   0.0278         Yes
6            Content Marketing       2.1214   0.0392         Yes
7   Content Marketing_Ad_Stock       3.5817   0.0009         Yes
8             Online marketing       3.1773   0.0028         Yes
9    Online marketing_Ad_Stock       3.9054   0.0004         Yes
10                  Affiliates       2.4608   0.0178         Yes
11         Affiliates_Ad_Stock       2.2830   0.0275         Yes
12                         SEM       2.1792   0.0351         Yes
13                SEM_Ad_Stock       2.1792   0.

In [7]:
import statsmodels.api as sm

X = df[channels]
y = df['gmv']

# Add constant term for intercept
X = sm.add_constant(X)

ols_model = sm.OLS(y, X).fit()
print("\nOLS Regression Summary:")
print(ols_model.summary())

# Extract coefficients
coefficients = ols_model.params.reset_index()
coefficients.columns = ['Channel', 'Coefficient']
print("\nCoefficients (Effect Size):")
print(coefficients)

OLS Regression Summary:
                            OLS Regression Results                             
Dep. Variable:                    gmv   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     2.400
Date:                Wed, 19 Mar 2025   Prob (F-statistic):             0.0147
Time:                        19:57:01   Log-Likelihood:                -942.57
No. Observations:                  51   AIC:                             1927.
Df Residuals:                      30   BIC:                             1968.
Df Model:                          20                                          
Covariance Type:            nonrobust                                          
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [8]:
spend_data = df[channels].sum()

roi_list = []
for channel in channels:
    impact = ols_model.params[channel]
    spend = spend_data[channel]
    roi = impact / spend if spend != 0 else 0
    roi_list.append({'Channel': channel, 'Impact_Coeff': impact, 'Total_Spend': spend, 'ROI': roi})

roi_df = pd.DataFrame(roi_list)
roi_df.sort_values(by='ROI', ascending=False, inplace=True)

print("\nROI for Each Channel:")
print(roi_df)



Radio: 1334006000.00
Affiliates: 40084510.00
Digital: 29342680.00
TV: 3112100.00
Affiliates_Ad_Stock: 1373886.00
SEM_Ad_Stock: 865596.30
Digital_Ad_Stock: 216470.30
Other_Ad_Stock: 160566.20
Sponsorship: 67294.14
NPS: 14999.20
Stock Index: -23.28
Sponsorship_Ad_Stock: -111.39
Online marketing_Ad_Stock: -56838.00
TV_Ad_Stock: -282872.40
SEM: -3487485.00
Online marketing: -3681992.00
Radio_Ad_Stock: -6248698.00
Content Marketing_Ad_Stock: -9515425.00
Other: -11678890.00
Content Marketing: -91419770.00


In [9]:
def objective_function(allocation):
    # Calculate predicted GMV based on allocation
    predicted_gmv = model.intercept_ + np.sum(allocation * model.coef_)
    # Calculate ROI as GMV / total investment
    roi = predicted_gmv / np.sum(allocation)
    # Return negative ROI since we want to maximize it (minimize negative)
    return -roi

# Define constraints
def constraint_total_budget(allocation):
    # Ensure total allocation equals current total investment
    return np.sum(allocation) - current_total_investment

# Initial guess - equal allocation
initial_guess = np.ones(len(marketing_channels)) * (current_total_investment / len(marketing_channels))

# Bounds for each channel - minimum 0, no upper bound
bounds = [(0, None) for _ in range(len(marketing_channels))]

# Constraints
constraints = [{'type': 'eq', 'fun': constraint_total_budget}]

# Run optimization
result = minimize(objective_function, initial_guess, method='SLSQP',
                  bounds=bounds, constraints=constraints)

# Get optimized allocation
optimized_allocation = result.x

# Calculate optimized ROI
optimized_gmv = model.intercept_ + np.sum(optimized_allocation * model.coef_)
optimized_roi = optimized_gmv / np.sum(optimized_allocation)
current_gmv = model.intercept_ + np.sum(current_allocation * model.coef_)
current_roi = current_gmv / current_total_investment

print("\nOptimization results:")
print(f"Optimized ROI: {-result.fun:.4f}")
print(f"Current ROI: {current_roi:.4f}")
print(f"ROI improvement: {-result.fun - current_roi:.4f} ({(-result.fun - current_roi)/current_roi*100:.2f}%)")

print("\nOptimized allocation:")
allocation_data = []
for i, channel in enumerate(marketing_channels):
    current_pct = current_allocation[channel]/current_total_investment*100
    optimized_pct = optimized_allocation[i]/current_total_investment*100
    change_pct = optimized_pct - current_pct
    print(f"{channel}: {optimized_allocation[i]:.2f} ({optimized_pct:.2f}%) | Change: {change_pct:+.2f}%")
    allocation_data.append({
        'Channel': channel,
        'Current': current_allocation[channel],
        'Optimized': optimized_allocation[i],
        'Current %': current_pct,
        'Optimized %': optimized_pct,
        'Change %': change_pct
    })

# Create a DataFrame for better reporting
allocation_df = pd.DataFrame(allocation_data)
allocation_df = allocation_df.sort_values('Optimized', ascending=False)

x = np.arange(len(marketing_channels))
width = 0.35

# Sort channels by optimized allocation for better visualization
sorted_indices = np.argsort(optimized_allocation)[::-1]
sorted_channels = [marketing_channels[i] for i in sorted_indices]
sorted_current = [current_allocation[channel] for channel in sorted_channels]
sorted_optimized = [optimized_allocation[i] for i, channel in enumerate(marketing_channels) if channel in sorted_channels]

# Advanced analysis: Sensitivity analysis
# Let's see how ROI changes with small changes in allocation
sensitivities = []
for i, channel in enumerate(marketing_channels):
    if optimized_allocation[i] > 0:
        # Create a baseline allocation equal to the optimized one
        baseline = optimized_allocation.copy()

        # Increase the allocation for this channel by 10%
        baseline[i] *= 1.1

        # Normalize to maintain total budget
        baseline = baseline * (current_total_investment / np.sum(baseline))

        # Calculate new ROI
        new_gmv = model.intercept_ + np.sum(baseline * model.coef_)
        new_roi = new_gmv / current_total_investment

        # Calculate sensitivity
        sensitivity = (new_roi - (-result.fun)) / (0.1 * optimized_allocation[i])
        sensitivities.append((channel, sensitivity))
    else:
        # Skip channels with zero allocation
        sensitivities.append((channel, 0))

print("\nSensitivity Analysis:")
for channel, sensitivity in sorted(sensitivities, key=lambda x: x[1], reverse=True):
    print(f"{channel}: {sensitivity:.6f}")

# Plot sensitivity analysis for channels with non-zero sensitivity
non_zero_channels = [channel for channel, sensitivity in sensitivities if sensitivity != 0]
non_zero_sensitivities = [sensitivity for channel, sensitivity in sensitivities if sensitivity != 0]

# Calculate expected GMV with optimized allocation
expected_gmv = model.intercept_ + np.sum(optimized_allocation * model.coef_)
print(f"\nExpected GMV with optimized allocation: {expected_gmv:.2f}")
print(f"Current GMV: {current_gmv:.2f}")
print(f"GMV improvement: {expected_gmv - current_gmv:.2f} ({(expected_gmv - current_gmv)/current_gmv*100:.2f}%)")

# Let's also examine diminishing returns by modeling non-linear relationships
# This approach uses polynomial features to capture diminishing returns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

print("\nModeling with diminishing returns:")
# Create polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X)

# Get feature names for the polynomial features
poly_features = []
for i, channel in enumerate(marketing_channels):
    poly_features.append(channel)

for i in range(len(marketing_channels)):
    for j in range(i, len(marketing_channels)):
        if i == j:
            poly_features.append(f"{marketing_channels[i]}^2")
        else:
            poly_features.append(f"{marketing_channels[i]}*{marketing_channels[j]}")

# Train a model with polynomial features
poly_model = LinearRegression()
poly_model.fit(X_poly, y)

# Define the objective function with polynomial features
def poly_objective_function(allocation):
    # Transform allocation using polynomial features
    allocation_poly = poly.transform(allocation.reshape(1, -1)).flatten()

    # Calculate predicted GMV based on allocation
    predicted_gmv = poly_model.intercept_ + np.sum(allocation_poly * poly_model.coef_)

    # Calculate ROI as GMV / total investment
    roi = predicted_gmv / np.sum(allocation)

    # Return negative ROI since we want to maximize it
    return -roi

# Run optimization with polynomial model
poly_result = minimize(poly_objective_function, initial_guess, method='SLSQP',
                      bounds=bounds, constraints=constraints)

# Get optimized allocation with polynomial model
poly_optimized_allocation = poly_result.x

# Calculate optimized ROI with polynomial model
poly_optimized_allocation_transformed = poly.transform(poly_optimized_allocation.reshape(1, -1)).flatten()
poly_optimized_gmv = poly_model.intercept_ + np.sum(poly_optimized_allocation_transformed * poly_model.coef_)
poly_optimized_roi = poly_optimized_gmv / np.sum(poly_optimized_allocation)

# Calculate current GMV and ROI with polynomial model
current_allocation_transformed = poly.transform(current_allocation.values.reshape(1, -1)).flatten()
poly_current_gmv = poly_model.intercept_ + np.sum(current_allocation_transformed * poly_model.coef_)
poly_current_roi = poly_current_gmv / current_total_investment

print("\nPolynomial model optimization results:")
print(f"Polynomial Optimized ROI: {-poly_result.fun:.4f}")
print(f"Polynomial Current ROI: {poly_current_roi:.4f}")
print(f"ROI improvement: {-poly_result.fun - poly_current_roi:.4f} ({(-poly_result.fun - poly_current_roi)/poly_current_roi*100:.2f}%)")

print("\nPolynomial optimized allocation:")
for i, channel in enumerate(marketing_channels):
    current_pct = current_allocation[channel]/current_total_investment*100
    optimized_pct = poly_optimized_allocation[i]/current_total_investment*100
    change_pct = optimized_pct - current_pct
    print(f"{channel}: {poly_optimized_allocation[i]:.2f} ({optimized_pct:.2f}%) | Change: {change_pct:+.2f}%")


sorted_indices = np.argsort(optimized_allocation)[::-1]
sorted_channels = [marketing_channels[i] for i in sorted_indices]
sorted_current = [current_allocation[channel] for channel in sorted_channels]
sorted_linear_opt = [optimized_allocation[i] for i, channel in enumerate(marketing_channels) if channel in sorted_channels]
sorted_poly_opt = [poly_optimized_allocation[i] for i, channel in enumerate(marketing_channels) if channel in sorted_channels]



Optimized Budget Allocation:
Radio: 299.78
Digital: 102.23
TV: 25.65
Online marketing: 9.52
Sponsorship: 7.72

Percentage Change in Allocation:
Radio: 8489.79%
Digital: 405.57%
TV: -11.51%
Online marketing: -95.01%
Sponsorship: -91.64%
